<a href="https://www.kaggle.com/code/abhishektiwari64/chatbot-using-rag?scriptVersionId=143633567" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Problem Statement: You are given a knowledge document over which you need to build a chatbot using Retrieval Augmented Generation (RAG). The RAG module should be capable of receiving user question as input and will need to output an answer to it based on the knowledge provided in the document

Let's install required libraries:

1. LangChain’s flexible abstractions and extensive toolkit enables developers to harness the power of LLMs.
2. Pinecone is a cloud-based vector database service that provides a managed and scalable platform for similarity search and vector indexing
3. openai provides with LLM
4. The "tiktoken" library is a tool developed by OpenAI that allows you to count the number of tokens in a text string without making an API call to OpenAI's servers

In [ ]:
pip install langchain

In [ ]:
!pip install pinecone-client

In [ ]:
!pip install openai

In [ ]:
!pip install tiktoken

In [ ]:
import langchain
import pinecone

# TextLoader will be used to load data from Pan_card_services.txt to the document 
from langchain.document_loaders import TextLoader

# CharacterTextSplitter will be used to chunkify the document
from langchain.text_splitter import CharacterTextSplitter

# OpenAIEmbeddings will be used to convert individual chunks and query to vectors
from langchain.embeddings.openai import OpenAIEmbeddings

#Pinecone serves as vector-database
from langchain.vectorstores import Pinecone

#OpenAI will provide with LLM to generate response to the query
from langchain import OpenAI

#The RetrievalQAChain is a chain that combines a Retriever and a QA chain. 
# It is used to retrieve documents from a Retriever and then use a QA chain to answer a question 
# based on the retrieved documents.
from langchain.chains import RetrievalQA

Let's load data from Pan_card_services.txt to the document 

In [ ]:
loader = TextLoader("../input/knowledgedocuments/Pan_card_services.txt")
document = loader.load()
print(document)

data is succefully loaded into the document

Let's chunkify(split document into chunks) the document

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=60)
texts = text_splitter.split_documents(document)
print(len(texts))

document is split into 100 chunks

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
openAi_api_key = user_secrets.get_secret("openAiKey")
pinecone_api_key = user_secrets.get_secret("pineconeKey")

# set the env variable in order to use openAI module
os.environ["OPENAI_API_KEY"] = openAi_api_key

pinecone.init(api_key=pinecone_api_key, environment="gcp-starter")

I will initialize OpenAIEmbeddings and create embeddings object to convert chunks into vectors

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=openAi_api_key)

following will take the chunks, convert them into vectors using embeddings(Embedding model) object, and store the vectors into Pinecone

In [ ]:
docsearch = Pinecone.from_documents(texts, embeddings, index_name="chatbot-rag-embeddings-index")

following will:
1. take query in prompt, convert query to vector and place this vector in vector database
2. then top k vectors semantically similar to query vector will chosen
3. top k vectors will be converted back to respective chunks and stuffed into prompt
4. new prompt containing these k chunks and query will be fed to openAi LLM to generate response

In [ ]:
qa = RetrievalQA.from_chain_type(llm = OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())

Now, we can ask questions and get answers

In [ ]:
query = "What is the cost/fees of a PAN card?"
result = qa({"query":query})
print(result["result"])

In [ ]:
query = "Can I apply for a PAN card if I am a non-resident Indian (NRI)?"
result = qa({"query":query})
print(result["result"])

In [ ]:
query = "How to apply for PAN card?"
result = qa({"query":query})
print(result["result"])